In [2]:
import pandas as pd

data = pd.read_csv('netflix_titles.csv')

data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
df = data[data['country'] == 'Nigeria']

In [7]:
ans = df[df['title'] == 'Castle and Castle'].loc[:,'description']
ans

49    A pair of high-powered, successful lawyers fin...
Name: description, dtype: object

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 49 to 8556
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       95 non-null     object
 1   type          95 non-null     object
 2   title         95 non-null     object
 3   director      90 non-null     object
 4   cast          93 non-null     object
 5   country       95 non-null     object
 6   date_added    95 non-null     object
 7   release_year  95 non-null     int64 
 8   rating        95 non-null     object
 9   duration      95 non-null     object
 10  listed_in     95 non-null     object
 11  description   95 non-null     object
dtypes: int64(1), object(11)
memory usage: 9.6+ KB


In [5]:
df.isnull().sum()

show_id         0
type            0
title           0
director        5
cast            2
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [6]:
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
df = df[['title', 'cast', 'description', 'type']]

In [9]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 49 to 8556
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        95 non-null     object
 1   cast         93 non-null     object
 2   description  95 non-null     object
 3   type         95 non-null     object
dtypes: object(4)
memory usage: 3.7+ KB


In [10]:
df.dropna(axis=0, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93 entries, 49 to 8556
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        93 non-null     object
 1   cast         93 non-null     object
 2   description  93 non-null     object
 3   type         93 non-null     object
dtypes: object(4)
memory usage: 3.6+ KB


In [12]:
df.set_index('title', inplace = True)

In [13]:
df.head()

,cast,description,type
title,,,
Castle and Castle,"Richard Mofe-Damijo, Dakore Akande, Bimbo Manu...","A pair of high-powered, successful lawyers fin...",TV Show
King of Boys,"Sola Sobowale, Adesua Etomi, Remilekun ""Remini...",When a powerful businesswoman’s political ambi...,Movie
Omo Ghetto: the Saga,"Funke Akindele, Ayo Makun, Chioma Chukwuka Akp...",Twins are reunited as a good-hearted female ga...,Movie
King of Boys: The Return of the King,"Sola Sobowale, Toni Tones, Richard Mofe-Damijo...",Alhaja Eniola Salami starts anew and sets her ...,TV Show
Brother Jekwu,"Mike Ezuruonye, Juliet Ochienge, Angela Okorie...","Chasing wild success, a village hustler follow...",Movie


In [14]:
df['bag_of_words'] = '' # This will contain all the words in all columns joined together
columns = df.columns

for index, row in df.iterrows():
    
    words = ''
   
    for col in columns:
        
        if col != 'type' and col != 'description' and col != 'cast':
            words = words + ' '.join(row[col])+' '
        else:
            words = words + row[col]+ ' '
            
    row['bag_of_words'] = words

In [15]:
df.head()

,cast,description,type,bag_of_words
title,,,,
Castle and Castle,"Richard Mofe-Damijo, Dakore Akande, Bimbo Manu...","A pair of high-powered, successful lawyers fin...",TV Show,"Richard Mofe-Damijo, Dakore Akande, Bimbo Manu..."
King of Boys,"Sola Sobowale, Adesua Etomi, Remilekun ""Remini...",When a powerful businesswoman’s political ambi...,Movie,"Sola Sobowale, Adesua Etomi, Remilekun ""Remini..."
Omo Ghetto: the Saga,"Funke Akindele, Ayo Makun, Chioma Chukwuka Akp...",Twins are reunited as a good-hearted female ga...,Movie,"Funke Akindele, Ayo Makun, Chioma Chukwuka Akp..."
King of Boys: The Return of the King,"Sola Sobowale, Toni Tones, Richard Mofe-Damijo...",Alhaja Eniola Salami starts anew and sets her ...,TV Show,"Sola Sobowale, Toni Tones, Richard Mofe-Damijo..."
Brother Jekwu,"Mike Ezuruonye, Juliet Ochienge, Angela Okorie...","Chasing wild success, a village hustler follow...",Movie,"Mike Ezuruonye, Juliet Ochienge, Angela Okorie..."


In [20]:
df['bag_of_words'].loc['Castle and Castle']

'Richard Mofe-Damijo, Dakore Akande, Bimbo Manuel, Blossom Chukwujekwu, Deyemi Okanlawon, Etim Effiong, Denola Grey, Duke Akintola, Eku Edewor, Ade Laoye, Anee Icha, Kevin Ushi, Jude Chukwuka, Amanda Afolabi A pair of high-powered, successful lawyers find themselves defending opposite interests of the justice system, causing a strain on their happy marriage. TV Show  '

In [21]:
cv = CountVectorizer() # Instantiating the countvectorizer object
cv_matrix = cv.fit_transform(df['bag_of_words'])

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/CodexJoe/Projects/main/movie_recommendation_system/netflix_titles.csv')
df = df[df['country'] == 'Nigeria']
df['rating'] = df['rating'].fillna(df['rating'].mode()[0])
df['duration'] = df['duration'].fillna(df['duration'].mode()[0])
df['date_added'] = df['date_added'].fillna(df['date_added'].mode()[0])
df['country'] = df['country'].fillna(df['country'].mode()[0])
df['director'] = df['director'].fillna('Unobtainable')
df['cast'] = df["cast"].fillna('Unobtainable')
movies = df.loc[:, ['title', 'type', 'director', 'cast', 'rating', 'listed_in', 'description']]

In [4]:
movies

,title,type,director,cast,rating,listed_in,description
49,Castle and Castle,TV Show,Unobtainable,"Richard Mofe-Damijo, Dakore Akande, Bimbo Manu...",TV-MA,"International TV Shows, TV Dramas","A pair of high-powered, successful lawyers fin..."
73,King of Boys,Movie,Kemi Adetiba,"Sola Sobowale, Adesua Etomi, Remilekun ""Remini...",TV-MA,"Dramas, International Movies",When a powerful businesswoman’s political ambi...
84,Omo Ghetto: the Saga,Movie,"JJC Skillz, Funke Akindele","Funke Akindele, Ayo Makun, Chioma Chukwuka Akp...",TV-MA,"Action & Adventure, Comedies, Dramas",Twins are reunited as a good-hearted female ga...
198,King of Boys: The Return of the King,TV Show,Kemi Adetiba,"Sola Sobowale, Toni Tones, Richard Mofe-Damijo...",TV-MA,"Crime TV Shows, International TV Shows, TV Dramas",Alhaja Eniola Salami starts anew and sets her ...
281,Brother Jekwu,Movie,Charles Uwagbai,"Mike Ezuruonye, Juliet Ochienge, Angela Okorie...",TV-14,"Comedies, International Movies","Chasing wild success, a village hustler follow..."
...,...,...,...,...,...,...,...
7875,Road to Yesterday,Movie,Ishaya Bako,"Genevieve Nnaji, Oris Erhuero, Majid Michel",TV-14,"Dramas, International Movies","Their marriage failing, Victoria and Izu try t..."
8227,The Bridge,Movie,Kunle Afolayan,"Chidinma Ekile, Ademola Adedoyin, Kunle Afolay...",TV-14,"Dramas, International Movies, Romantic Movies",A Yoruba prince and a young lady from a promin...
8241,The CEO,Movie,Kunle Afolayan,"Wale Ojo, Jimmy Jean-Louis, Hilda Dokubo, Nico...",TV-14,"Dramas, International Movies, Thrillers",A group of executives heads to a telecommunica...
8297,The Figurine (Araromire),Movie,Kunle Afolayan,"Ramsey Nouah, Omoni Oboli, Kunle Afolayan, Fun...",TV-14,"Horror Movies, International Movies, Thrillers",When a pair of friends discovers a mystical sc...
